## Benchmark Point in Polygon

Some realistic test cases of enriching polygons with point data and vice versa, using Uber's NYC Trip Data and census blocks.

Inspired by this post:

https://www.linkedin.com/posts/mbforr_python-sql-spatialsql-activity-6834177271616589824-ScB-

In [103]:
con = omnisci_connect()
con

#### Spatial Join Timing Test

##### Enriching uber points with census ids

In [106]:
q = "SELECT "
q +=    "trips.trip_datetime, trips.base, trips.omnisci_geo, "
q +=    "census.STATE_FIPS, census.CNTY_FIPS, census.STCOFIPS, census.TRACT, census.BLKGRP, census.FIPS  "
q += "FROM "
q +=    "uber_nyc_trips_geo as trips, us_census_bg as census "
q += "WHERE "
q +=   f"ST_CONTAINS(census.omnisci_geo, trips.omnisci_geo)"
q

'SELECT trips.trip_datetime, trips.base, trips.omnisci_geo, census.STATE_FIPS, census.CNTY_FIPS, census.STCOFIPS, census.TRACT, census.BLKGRP, census.FIPS  FROM uber_nyc_trips_geo as trips, us_census_bg as census WHERE ST_CONTAINS(census.omnisci_geo, trips.omnisci_geo)'

In [107]:
t0 = time()
con.con.execute(q)
t1 = time()
print(f"Spatial enrichment of 4.5m points in memory with geo column took {t1-t0} seconds")

KeyboardInterrupt: 

In [99]:
ctas = f"CREATE TABLE IF NOT EXISTS uber_nyc_census_enriched_geo AS ({q})"
ctas

'CREATE TABLE IF NOT EXISTS uber_nyc_census_enriched_geo AS (SELECT trips.trip_datetime, trips.base, trips.omnisci_geo, census.STATE_FIPS, census.CNTY_FIPS, census.STCOFIPS, census.TRACT, census.BLKGRP, census.FIPS  FROM uber_nyc_trips_geo as trips, us_census_bg as census WHERE ST_CONTAINS(census.omnisci_geo, trips.omnisci_geo))'

In [100]:
t0 = time()
con.con.execute(ctas)
t1 = time()
print(f"Spatial enrichment of 4.5m points with materialization took {t1-t0} seconds")

KeyboardInterrupt: 

Using raw longitude and latitude columns

In [75]:
trip_points = "ST_SETSRID(ST_POINT(trips.longitude, trips.latitude),4326)"
q = "SELECT "
q +=    "trips.trip_datetime, trips.base, "
q +=   f"{trip_points} as omnisci_geo, "
q +=    "census.STATE_FIPS, census.CNTY_FIPS, census.STCOFIPS, census.TRACT, census.BLKGRP, census.FIPS  "
q += "FROM "
q +=    "uber_nyc_trips as trips, us_census_bg as census "
q += "WHERE "
q +=   f"ST_CONTAINS(census.omnisci_geo, {trip_points})"
q

'SELECT trips.trip_datetime, trips.base, ST_SETSRID(ST_POINT(trips.longitude, trips.latitude),4326) as omnisci_geo, census.STATE_FIPS, census.CNTY_FIPS, census.STCOFIPS, census.TRACT, census.BLKGRP, census.FIPS  FROM uber_nyc_trips as trips, us_census_bg as census WHERE ST_CONTAINS(census.omnisci_geo, ST_SETSRID(ST_POINT(trips.longitude, trips.latitude),4326))'

In [77]:
t0 = time()
con.con.execute(q)
t1 = time()
print(f"Spatial enrichment of 4.5m points in memory with raw columns took {t1-t0} seconds")

Spatial enrichment of 4.5m points in memory took 40.114715337753296 seconds


In [72]:
con.con.execute("DROP TABLE IF EXISTS uber_nyc_census_enriched")

In [73]:
ctas = f"CREATE TABLE IF NOT EXISTS uber_nyc_census_enriched AS ({q})"

In [74]:
t0 = time()
con.con.execute(ctas)
t1 = time()
print(f"Spatial enrichment of 4.5m points with materialization took {t1-t0} seconds")

Spatial enrichment of 4.5m points with materialization took 13.538450241088867 seconds


#### Summarization on Block Group Geometries

In [45]:
trip_points = "ST_SETSRID(ST_POINT(trips.longitude, trips.latitude),4326)"
q = "SELECT "
q +=    "count(*) as trip_count, count(distinct trips.base) as base_count, "
q +=   f"sample(census.omnisci_geo) as omnisci_geo, "
q +=    "sample(census.STATE_FIPS) as STATE_FIPS, "
q +=    "sample(census.CNTY_FIPS) as CNTY_FIPS, "
q +=    "sample(census.STCOFIPS) as STCOFIPS, "
q +=    "sample(census.TRACT) as TRACT, "
q +=    "sample(census.BLKGRP) as BLKGRP, "
q +=    "census.FIPS  "
q += "FROM "
q +=    "uber_nyc_trips as trips, us_census_bg as census "
q += "WHERE "
q +=   f"ST_CONTAINS(census.omnisci_geo, {trip_points})"
q += "GROUP BY "
q +=   "census.FIPS"
q

'SELECT count(*) as trip_count, count(distinct trips.base) as base_count, sample(census.omnisci_geo) as omnisci_geo, sample(census.STATE_FIPS) as STATE_FIPS, sample(census.CNTY_FIPS) as CNTY_FIPS, sample(census.STCOFIPS) as STCOFIPS, sample(census.TRACT) as TRACT, sample(census.BLKGRP) as BLKGRP, census.FIPS  FROM uber_nyc_trips as trips, us_census_bg as census WHERE ST_CONTAINS(census.omnisci_geo, ST_SETSRID(ST_POINT(trips.longitude, trips.latitude),4326))GROUP BY census.FIPS'

In [50]:
t0 = time()
con.con.execute(q)
t1 = time()
print(f"Spatial enrichment of census block polygons from 4.5m points in memory took {t1-t0} seconds")

Spatial enrichment of census block polygons from 4.5m points in memory took 4.182528495788574 seconds


In [55]:
con.con.execute("DROP TABLE IF EXISTS nyc_census_bg_uber_enriched")

In [56]:
ctas = f"CREATE TABLE IF NOT EXISTS nyc_census_bg_uber_enriched AS ({q})"

In [57]:
t0 = time()
con.con.execute(ctas)
t1 = time()
print(f"Spatial enrichment of census blocks with 4.5m points with materialization took {t1-t0} seconds")

Spatial enrichment of census blocks with 4.5m points with materialization took 5.210094928741455 seconds
